In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir("./drive/MyDrive/BOAZ ADV/")

# Rating

In [ ]:
!pip install konlpy

In [ ]:
from datetime import datetime
import numpy as np
import glob
import pandas as pd
import re
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
import time
import json
from tensorflow.keras.preprocessing.sequence import pad_sequences
from konlpy.tag import Okt
from gensim.models import FastText
from text_preprocess import apply_model,saved_model

In [ ]:
platform_list = ["naver_crawling","kakao_crawling","google_crawling"]
naver_list = os.path.join(os.getcwd(),platform_list[0])
kakao_list = os.path.join(os.getcwd(),platform_list[1])
google_list = os.path.join(os.getcwd(),platform_list[2])

In [ ]:
def to_date(string): 
  try:
    x = pd.to_datetime(string,format="%Y.%m.%d").date()
  except:
    x = np.nan
  return x

In [ ]:
# rating 
rating = pd.DataFrame()
targetPattern = os.path.join(naver_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp["Platform"] = "Naver"
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]
rating["Timestamp"] = rating["Timestamp"].apply(to_date)
rating.dropna(inplace=True)

In [ ]:
targetPattern = os.path.join(kakao_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp.dropna(inplace=True)
  temp["Platform"] = "Kakao"
  temp["Timestamp"] = temp["Timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
  temp["Rating"] = temp["Rating"].apply(lambda x : float(x[0]))
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]

구글 크롤링 시 Timestamp가 날짜가 아닌 며칠 전, 몇 달 전, 몇 년 전으로 데이터 수집<br>
=> 크롤링 날짜를 기준으로 일괄적으로 처리

In [ ]:
my_date = datetime(2021, 4, 30) # 크롤링 날짜로 변경

def change_datetime(input):
    str_list = ['년', '달', '주', '일', '시간']
    p_num = re.compile('[0-9]')
    p_non_num = re.compile('[^0-9]')
    s=""
    n=""
    date_str = input[:-2]

    
    # 숫자 추출
    n_list = p_num.findall(date_str)
    if len(n_list)==1:
        n = int(n_list[0])
    else:
        n_str=""
        for i in n_list:
            n_str = n_str+i
        n = str(n_str)
    
    # 문자 추출
    s_list = p_non_num.findall(date_str)
    if len(s_list)==1:
        s= s_list[0]
    else:
        s_str=""
        for i in s_list:
            s_str = s_str+i
        s = s_str
    n = int(n)
    #  문자에 따라 매핑
    if s == '년':
        tmp_date = my_date + relativedelta(years= (-1)*n)
        new_date = datetime(tmp_date.year, 1, 1) #1월1일 기준
    elif s == '달':
        tmp_date = my_date +  relativedelta(months= (-1)*n)     
        new_date = datetime(tmp_date.year, tmp_date.month, 1) #1일 기준
    elif s == '주':
        new_date = my_date +  relativedelta(days= (-1)*n*7)
    elif s== '일':
        new_date = my_date +  relativedelta(days= (-1)*n)
    elif s=='시간':
        new_date = my_date
    else:
        new_date = my_date
 
    return time.mktime(new_date.timetuple())


In [ ]:
targetPattern = os.path.join(google_list,"rating_*.csv")
for file_name in glob.glob(targetPattern):
  temp = pd.read_csv(file_name)
  temp["Platform"] = "Google"
  temp['Timestamp'] = temp.apply(lambda x : change_datetime(x['Timestamp']), axis=1)
  temp["Timestamp"] = temp["Timestamp"].apply(lambda x : datetime.fromtimestamp(x).date())
  rating = pd.concat([rating,temp])
  
rating = rating[["UserID","ItemID","Rating","Timestamp","Platform"]]
rating.reset_index(drop=True,inplace=True)

Rating 데이터 통합 완료

In [ ]:
print(f"Interaction Number = {len(rating)}")
display(rating.head())
display(rating.tail())

Interaction Number = 299179


,UserID,ItemID,Rating,Timestamp,Platform
0,5efdcc02c8819e06c65f1cbb,역삼역 반지만들기반지대학,5.0,2021-04-24,Naver
1,5c68d00f6f1dd1a8c5b61a64,역삼역 반지만들기반지대학,5.0,2021-04-15,Naver
2,5e9fdb798f87a842bc589744,역삼역 반지만들기반지대학,5.0,2021-04-03,Naver
3,5cc6f9192310d155e0d237dc,역삼역 반지만들기반지대학,5.0,2021-04-03,Naver
4,5ea2e3198f87a842bc0d8925,역삼역 반지만들기반지대학,5.0,2021-03-16,Naver


,UserID,ItemID,Rating,Timestamp,Platform
299174,100862155288500717490,녹사평역 올모스트홈카페,3.0,2020-01-01,Google
299175,103097387455202389670,녹사평역 올모스트홈카페,5.0,2020-01-01,Google
299176,107727968804299898351,녹사평역 올모스트홈카페,4.0,2020-05-01,Google
299177,106465249783626513449,녹사평역 올모스트홈카페,4.0,2020-07-01,Google
299178,117679202642374272134,상수역 지유명차,5.0,2020-12-01,Google


# UserID - ReviewID  

UserID와 ReviewID가 달라 생기는 문제를 방지하기 위해 매핑 작업 필요

In [ ]:
# Naver
user_review_json = {}
targetPattern = os.path.join(naver_list,"user_review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    user_review_json.update(temp)

In [ ]:
# Kakao
# 카카오는 존재 X

In [ ]:
# Google
targetPattern = os.path.join(google_list,"user_review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    user_review_json.update(temp)

In [ ]:
count = 0
for cafe in rating["ItemID"].unique():
  try:
    user = user_review_json[cafe]
  except:
    continue
  for u in user:
    try:
      temp = user[u]
      count+=1
    except:
      continue
print(f"UserID-ReviewID : {count}")

UserID-ReviewID : 160455


# Review Connect

In [ ]:
# Naver
review_json = {}
targetPattern = os.path.join(naver_list,"review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    review_json.update(temp)


In [ ]:
# Kakao
targetPattern = os.path.join(kakao_list,"review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"rb") as json_file:
    temp = json.load(json_file)
    review_json.update(temp)

In [ ]:
# Google
targetPattern = os.path.join(google_list,"review_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    review_json.update(temp)


In [ ]:
review = pd.DataFrame()
for key, value in review_json.items():
  try:
    temp = pd.DataFrame.from_dict(review_json[key], orient="index").reset_index()
  
    temp.columns = ["ReviewID","Review"]
    temp["ItemID"] = key
    review = pd.concat([temp,review])
  except:
    pass
review.reset_index(drop=True,inplace=True)
review = review[review["Review"].isnull() == False]

간단한 불용어 처리 뒤 벡터로 저장하는 코드<br>
FastText를 활용

In [ ]:
okt = Okt() 
review['Review'] = review["Review"].apply(lambda x : re.sub(pattern = '([ㄱ-ㅎㅏ-ㅣ]+)',repl=' ',string=str(x)) )
review["Review"] = review["Review"].str.replace(pat=r'[^\w]', repl=r' ', regex=True)
review['Review_token'] = review['Review'].apply(lambda x : okt.morphs(x))
ft_model = FastText(sentences = list(review['Review_token'].values), size=128, window=3, min_count=1) #변환되는 벡터 크기 : (size, )
review['Review_vector'] = review['Review'].apply(lambda x : apply_model(x,ft_model))
review.drop("Review_token",axis=1,inplace=True)
ft_model.save('./data/fasttext_model')

In [ ]:
review.to_csv("./data/review.csv",index=False)

In [ ]:
review

,ReviewID,Review,ItemID,Review_vector
0,ChZDSUhNMG9nS0VJQ0FnSURjaFp1RFJ3EAE,커피 맛이 좋아요 레드곰발 블랙곰발,신내역 곰발커피,"[-0.11765587, -0.14032328, 0.006572773, -0.020..."
1,ChdDSUhNMG9nS0VJQ0FnSUR5eXJ1Zmh3RRAB,커피맛도좋고 원두도 좋아요 첫 오픈때부터 지금까지 쭉 좋아요 오미자도 맛있고 여기...,신내역 곰발커피,"[-0.027292222, -0.0602496, 0.005346512, 0.0178..."
2,ChZDSUhNMG9nS0VJQ0FnSUR5MnRIRmFREAE,최고에용 직원분들도 넘 친절하구 원두도 다양해서 골라먹는 재미가 있습니당,신내역 곰발커피,"[-0.07474883, -0.1454044, -0.059100445, 0.0381..."
3,ChZDSUhNMG9nS0VJQ0FnSUR5MEktVERnEAE,커피 원두가 좋아요 아늑한 장소에요,신내역 곰발커피,"[-0.1768079, -0.21100509, 0.03473124, -0.00044..."
4,ChZDSUhNMG9nS0VJQ0FnSUNpci1MX013EAE,커피 속에 곰의 발의 기운이 들어가서 그런지 오후 내내 열심히 일했어요 퇴근하고 ...,신내역 곰발커피,"[-0.024490988, -0.08752858, -0.04979792, -0.09..."
...,...,...,...,...
170425,ChdDSUhNMG9nS0VJQ0FnSUNvZ1B2RGdBRRAB,커플 반지 만들 때 갔었어요,역삼역 반지만들기반지대학,"[0.06310286, -0.53482455, -0.7903507, -0.33368..."
170426,ChZDSUhNMG9nS0VJQ0FnSUN3MHAtZkJREAE,체험하고싶었는데 사장님이다해주셔서아쉬웠어요,역삼역 반지만들기반지대학,"[0.039604966, -0.19247445, -0.13229011, -0.123..."
170427,ChZDSUhNMG9nS0VJQ0FnSURTeTU2M053EAE,Google 번역 제공 우리는 학위를 얻었다 원문 we got degrees,역삼역 반지만들기반지대학,"[-0.22169106, -0.075879514, -0.32347494, 0.318..."
170428,ChdDSUhNMG9nS0VJQ0FnSURBeWIzVDh3RRAB,Google 번역 제공 실버로 나만의 반지를 만들 수있는 카페 링 메이킹 과정...,역삼역 반지만들기반지대학,"[-0.08500654, -0.039808806, -0.100377664, 0.08..."


# Image Connect

In [ ]:
# Naver
image_json = {}
targetPattern = os.path.join(naver_list,"image_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    image_json.update(temp)


In [ ]:
# Kakao
# 6호선 이미지만 수집 중
targetPattern = os.path.join(kakao_list,"image_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    image_json.update(temp)


In [ ]:
# Google
targetPattern = os.path.join(google_list,"image_*.json")
for file_name in glob.glob(targetPattern):
  with open(file_name,"r") as json_file:
    temp = json.load(json_file)
    image_json.update(temp)

In [ ]:
image= pd.DataFrame()
for key, value in image_json.items():
  try:
    temp = pd.DataFrame.from_dict(image_json[key], orient="index").reset_index()
  
    temp.columns = ["ReviewID","Image"]
    image = pd.concat([temp,image])
  except:
    pass
image.reset_index(drop=True, inplace=True)

In [ ]:
image.to_csv("./data/image.csv",index=False)

# Mapping

In [ ]:
def user2reviewid(row):
  try:
    user_review = user_review_json.get(row["ItemID"])
    user_review = user_review.get(row["UserID"])
    return user_review
  except:
    return row["UserID"]

def user2review(row):
  try:
    review = review_json.get(row["ItemID"])
    review = review.get(row["ReviewID"])
    return review
  except:
    return "리뷰 없음"

def user2image(row):
  try:
    image = image_json.get(row["ItemID"])
    image = image.get(row["ReviewID"])
    if image != "":
      return f"{row['ItemID']}_{row['ReviewID']}.jpg"
    else:
      return "이미지 없음"
  except:
    return "이미지 없음"

In [ ]:
rating["ReviewID"] = rating.apply(lambda x : user2reviewid(x),axis=1)
rating.loc[rating["ReviewID"].isnull(),"ReviewID"] = "Not Review ID"
rating.reset_index(drop=True,inplace=True)

성별, 선호도, 설문조사 같은 수집할 수 없는 데이터를 처리하기 위해 임의의 값으로 초기화

In [ ]:
# 데이터 쌓이면 get dummy
rating["Gender_F"] = 0
rating["Gender_M"] = 0
rating["Gender_X"] = 1

rating["Favorite"] = 0

rating["Survey"] = np.nan


In [ ]:
#data = pd.DataFrame({"Survey":["분위기","분위기,맛,서비스","서비스,맛,가격",np.nan]})
survey2index = {"분위기":1,"맛":2,"서비스":3,"접근성":4,"가격":5}
def survey_pre(x):
  #temp = np.zeros(5)
  try:
    x = x.split(",")
    x = [survey2index.get(xx) for xx in x]
  except:
    x = [0]
  return x
rating["Survey"]= rating["Survey"].apply(lambda x : survey_pre(x))
temp = pad_sequences(rating["Survey"],maxlen=5,padding="post")
temp = pd.DataFrame(temp,columns = ["Survey_1","Survey_2","Survey_3","Survey_4","Survey_5"])
rating = pd.concat([rating,temp],axis=1)
rating.drop("Survey",axis=1,inplace=True)

In [ ]:
rating

,UserID,ItemID,Rating,Timestamp,Platform,ReviewID,Gender_F,Gender_M,Gender_X,Favorite,Survey_1,Survey_2,Survey_3,Survey_4,Survey_5
0,5efdcc02c8819e06c65f1cbb,역삼역 반지만들기반지대학,5.0,2021-04-24,Naver,Not Review ID,0,0,1,0,0,0,0,0,0
1,5c68d00f6f1dd1a8c5b61a64,역삼역 반지만들기반지대학,5.0,2021-04-15,Naver,Not Review ID,0,0,1,0,0,0,0,0,0
2,5e9fdb798f87a842bc589744,역삼역 반지만들기반지대학,5.0,2021-04-03,Naver,Not Review ID,0,0,1,0,0,0,0,0,0
3,5cc6f9192310d155e0d237dc,역삼역 반지만들기반지대학,5.0,2021-04-03,Naver,Not Review ID,0,0,1,0,0,0,0,0,0
4,5ea2e3198f87a842bc0d8925,역삼역 반지만들기반지대학,5.0,2021-03-16,Naver,Not Review ID,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299174,100862155288500717490,녹사평역 올모스트홈카페,3.0,2020-01-01,Google,ChdDSUhNMG9nS0VJQ0FnSUNjLV9PMF9BRRAB,0,0,1,0,0,0,0,0,0
299175,103097387455202389670,녹사평역 올모스트홈카페,5.0,2020-01-01,Google,ChZDSUhNMG9nS0VJQ0FnSUNjajVfbGNREAE,0,0,1,0,0,0,0,0,0
299176,107727968804299898351,녹사평역 올모스트홈카페,4.0,2020-05-01,Google,ChZDSUhNMG9nS0VJQ0FnSUQ4MTduWVBREAE,0,0,1,0,0,0,0,0,0
299177,106465249783626513449,녹사평역 올모스트홈카페,4.0,2020-07-01,Google,ChdDSUhNMG9nS0VJQ0FnSURDeGY2dDVRRRAB,0,0,1,0,0,0,0,0,0


Train Test Split
<br> => 이용자가 방문한 카페 중 가장 최근 카페를 Test 데이터로 지정
<br> Train에는 있지만 Test에는 없는 이용자나 카페, Test에는 있지만 Train에는 없는 이용자나 카페는 모두 제거

In [ ]:
id_filter = rating.groupby("UserID").count()
id_filter = id_filter[id_filter["ItemID"] > 1].index
rating_filter = rating[rating["UserID"].isin(id_filter)]
test = rating_filter.sort_values('Timestamp').groupby('UserID').tail(1)
train = rating_filter[rating_filter.index.isin(test.index) == False]

test = test[(test["ItemID"].isin(set(train["ItemID"]))) & (test["UserID"].isin(set(train["UserID"])))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[(test["ItemID"].isin(set(train["ItemID"]))) & (test["UserID"].isin(set(train["UserID"])))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[(test["ItemID"].isin(set(train["ItemID"]))) & (test["UserID"].isin(set(train["UserID"])))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[(test["ItemID"].isin(set(train["ItemID"]))) & (test["UserID"].isin(set(train["UserID"])))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]
test = test[(test["ItemID"].isin(set(train["ItemID"]))) & (test["UserID"].isin(set(train["UserID"])))]
train = train[(train["ItemID"].isin(set(test["ItemID"]))) & (train["UserID"].isin(set(test["UserID"])))]

In [ ]:
print(train["ItemID"].nunique())
print(test["ItemID"].nunique())
print(len(set(test["ItemID"]) - set(train["ItemID"])))

print(train["UserID"].nunique())
print(test["UserID"].nunique())
print(len(set(test["UserID"]) - set(train["UserID"])))
print(len(set(train["UserID"]) - set(test["UserID"])))

7503
7503
0
53340
53340
0
0


# Cafe 7503개
# User 53340명

이용자와 카페를 모델에 집어넣기 위해 int 형태의 새로운 인덱스값 필요

In [ ]:
user_dict = {}
user_reverse_dict = {}
item_dict = {}
item_reverse_dict = {}


for i,q in enumerate(train["UserID"].unique()):
  user_dict[q] = i
  user_reverse_dict[i] = q
for i,q in enumerate(train["ItemID"].unique()):
  item_dict[q] = i
  item_reverse_dict[i] = q

In [ ]:
train["UserID_Re"] = train["UserID"].apply(lambda x : user_dict.get(x))
test["UserID_Re"] = test["UserID"].apply(lambda x : user_dict.get(x))

train["ItemID_Re"] = train["ItemID"].apply(lambda x : item_dict.get(x))
test["ItemID_Re"] = test["ItemID"].apply(lambda x : item_dict.get(x))

In [ ]:
train.reset_index(drop=True,inplace=True)
train.to_csv('./data/train_integration.csv')

In [ ]:
test.reset_index(drop=True,inplace=True)
test.to_csv('./data/test_integration.csv')